In [2]:
import boto3
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt

from sagemaker import Session

# Initialize a SageMaker session
sagemaker_session = Session()



role = "arn:aws:iam::016114370410:role/tf-binding-sites"
prefix = "inference/output/Motifs-AR-SRR12455436-2024-10-30-09-46-21"
local_dir = "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output"


# Initialize the S3 client
s3 = boto3.client('s3')

# Specify your S3 bucket name
bucket_name = sagemaker_session.default_bucket()


# Function to download files from an S3 bucket
def download_from_s3(bucket_name, prefix, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # List objects within the specified prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Iterate over each object
    for obj in response.get('Contents', []):
        key = obj['Key']
        # Skip if it's a folder
        if key.endswith('/'):
            continue
        local_path = os.path.join(local_dir, os.path.basename(key))
        
        # Download the file
        s3.download_file(bucket_name, key, local_path)
        print(f'Downloaded {key} to {local_path}')

# Bucket name extracted from the S3 URI
bucket_name = "tf-binding-sites"

# delete the files in local dir
for file in os.listdir(local_dir):
    os.remove(os.path.join(local_dir, file))
    

# Download the files
download_from_s3(bucket_name, prefix, "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output")


# Path to the directory containing JSON files
json_dir = "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output"

# Get a list of all JSON files in the directory
json_files = glob.glob(f"{json_dir}/*.jsonl.gz.out")

# Read each JSON file into a DataFrame and store in a list
dataframes = [pd.read_json(file) for file in json_files]

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

df

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /data1/home/wjarrard/.config/sagemaker/config.yaml
Downloaded inference/output/Motifs-AR-SRR12455436-2024-10-30-09-46-21/dataset_1.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_1.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455436-2024-10-30-09-46-21/dataset_10.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_10.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455436-2024-10-30-09-46-21/dataset_11.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_11.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455436-2024-10-30-09-46-21/dataset_12.jsonl.gz.out to /data1/datasets_1/human_

In [5]:
# count number of predictions in each class (1 and 0) in predicted column store as predicted if greater than .99 store as 1 else 0
df['predicted'] = (df['probabilities'] > 0.99).astype(int)
df['predicted'].value_counts()


predicted
0    50776
1    11380
Name: count, dtype: int64

In [6]:
import boto3
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt

from sagemaker import Session

# Initialize a SageMaker session
sagemaker_session = Session()



role = "arn:aws:iam::016114370410:role/tf-binding-sites"
prefix = "inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59"
local_dir = "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output"


# Initialize the S3 client
s3 = boto3.client('s3')

# Specify your S3 bucket name
bucket_name = sagemaker_session.default_bucket()


# Function to download files from an S3 bucket
def download_from_s3(bucket_name, prefix, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # List objects within the specified prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Iterate over each object
    for obj in response.get('Contents', []):
        key = obj['Key']
        # Skip if it's a folder
        if key.endswith('/'):
            continue
        local_path = os.path.join(local_dir, os.path.basename(key))
        
        # Download the file
        s3.download_file(bucket_name, key, local_path)
        print(f'Downloaded {key} to {local_path}')

# Bucket name extracted from the S3 URI
bucket_name = "tf-binding-sites"

# delete the files in local dir
for file in os.listdir(local_dir):
    os.remove(os.path.join(local_dir, file))
    

# Download the files
download_from_s3(bucket_name, prefix, "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output")


# Path to the directory containing JSON files
json_dir = "/data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output"

# Get a list of all JSON files in the directory
json_files = glob.glob(f"{json_dir}/*.jsonl.gz.out")

# Read each JSON file into a DataFrame and store in a list
dataframes = [pd.read_json(file) for file in json_files]

# Concatenate all DataFrames into a single DataFrame
df_neuro = pd.concat(dataframes, ignore_index=True)

df_neuro

Downloaded inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59/dataset_1.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_1.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59/dataset_10.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_10.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59/dataset_11.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_11.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59/dataset_12.jsonl.gz.out to /data1/datasets_1/human_cistrome/chip-atlas/peak_calls/tfbinding_scripts/tf-binding/data/jsonl_output/dataset_12.jsonl.gz.out
Downloaded inference/output/Motifs-AR-SRR12455437-2024-10-30-11-00-59/dataset_13.jsonl.gz.out to /data1/datase

,chr_name,start,end,cell_line,targets,predicted,weights,probabilities,linear_512_output,motifs
0,chr2,200510001,200510758,SRR12455437,-1,0,-1,0.000167,"[[-8.9021263123, -9.4344711304, -8.9212226868,...",GTGTACAAAAGTGTT
1,chr3,66154097,66154474,SRR12455437,-1,1,-1,0.664491,"[[1.9473198651999999, 1.3031423092, 1.30842316...",GATGGCTGAAGTTTG
2,chr1,208498107,208498366,SRR12455437,-1,0,-1,0.015885,"[[-4.1596803665, -6.5961031914, -6.450594902, ...",GTGCTCACATAGGTC
3,chr3,43758878,43759609,SRR12455437,-1,1,-1,0.997741,"[[5.09126091, 4.8833379745, 4.990278244, 4.758...",AGGCGAGGATGAGGG
4,chr4,129341639,129341877,SRR12455437,-1,0,-1,0.134914,"[[-2.3680477142, -4.2898578644, -4.1657195091,...",CATTTGTAGAAAAAG
...,...,...,...,...,...,...,...,...,...,...
46895,chr3,49028903,49029602,SRR12455437,-1,0,-1,0.000185,"[[-5.912981987, -6.6556963921, -6.1632127762, ...",TAGCACATAGTGGCC
46896,chr6,41733519,41734815,SRR12455437,-1,0,-1,0.169645,"[[0.5071838498, 0.5112509131, 0.5410997272, 0....",ATAGTTCCTGGCCGG
46897,chr1,160652243,160652727,SRR12455437,-1,0,-1,0.076931,"[[-6.150188446, -7.4227995872, -7.1572408676, ...",CAGCCCTTAATCCAT
46898,chr3,179604394,179605214,SRR12455437,-1,1,-1,0.932473,"[[0.7265093923, -0.3109875917, -0.0907084569, ...",CGGGACAAGTTGTGG


In [7]:
df_neuro['predicted'] = (df_neuro['probabilities'] > 0.999).astype(int)
df_neuro['predicted'].value_counts()


predicted
0    46613
1      287
Name: count, dtype: int64

In [1]:
df['predicted'] = (df['probabilities'] > 0.999).astype(int)
df['predicted'].value_counts()


NameError: name 'df' is not defined